In [3]:
# 1. Sagemaker
# 2. iam user

In [4]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import os
import os

sm_boto3=boto3.client('sagemaker')
session = sagemaker.Session()
region=session.boto_session.region_name
bucket="threat-matrix-sagemaker-bucket"

print("using bucket",bucket)

[04/17/25 20:01:07] INFO     Found credentials in shared    credentials.py:1352
                             credentials file:                                 
                             ~/.aws/credentials                                
using bucket threat-matrix-sagemaker-bucket


In [5]:
print(region)

us-east-1


In [6]:
df=pd.read_csv("phisingData.csv")
df.head()

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [7]:
df.isna().sum()

having_IP_Address              0
URL_Length                     0
Shortining_Service             0
having_At_Symbol               0
double_slash_redirecting       0
Prefix_Suffix                  0
having_Sub_Domain              0
SSLfinal_State                 0
Domain_registeration_length    0
Favicon                        0
port                           0
HTTPS_token                    0
Request_URL                    0
URL_of_Anchor                  0
Links_in_tags                  0
SFH                            0
Submitting_to_email            0
Abnormal_URL                   0
Redirect                       0
on_mouseover                   0
RightClick                     0
popUpWidnow                    0
Iframe                         0
age_of_domain                  0
DNSRecord                      0
web_traffic                    0
Page_Rank                      0
Google_Index                   0
Links_pointing_to_page         0
Statistical_report             0
Result    

In [8]:
features=list(df.columns)
features

['having_IP_Address',
 'URL_Length',
 'Shortining_Service',
 'having_At_Symbol',
 'double_slash_redirecting',
 'Prefix_Suffix',
 'having_Sub_Domain',
 'SSLfinal_State',
 'Domain_registeration_length',
 'Favicon',
 'port',
 'HTTPS_token',
 'Request_URL',
 'URL_of_Anchor',
 'Links_in_tags',
 'SFH',
 'Submitting_to_email',
 'Abnormal_URL',
 'Redirect',
 'on_mouseover',
 'RightClick',
 'popUpWidnow',
 'Iframe',
 'age_of_domain',
 'DNSRecord',
 'web_traffic',
 'Page_Rank',
 'Google_Index',
 'Links_pointing_to_page',
 'Statistical_report',
 'Result']

In [9]:
label=features.pop(-1)
label

'Result'

In [10]:
x = df.drop(columns=[label])
y = df[label]


In [11]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape) 

(8844, 30)
(2211, 30)
(8844,)
(2211,)


In [13]:
trainX=pd.DataFrame(X_train)
trainX[label]=y_train

testX=pd.DataFrame(X_test)
testX[label]=y_test

In [14]:
trainX

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
480,1,-1,1,1,1,-1,1,1,-1,1,1,1,1,1,1,-1,1,1,0,1,1,1,1,1,1,1,-1,1,0,1,1
10812,-1,-1,1,1,1,-1,0,1,1,1,1,1,-1,0,-1,-1,1,1,0,1,1,1,1,1,1,0,-1,-1,0,1,-1
4064,1,1,1,1,1,1,0,1,-1,1,1,1,1,0,-1,1,1,1,0,1,1,1,1,-1,1,0,-1,-1,0,1,1
8225,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,1,0,0,0,1,-1,0,1,1,1,1,1,-1,0,-1,-1,1,1,1
9432,-1,-1,1,1,1,-1,-1,-1,-1,1,1,-1,-1,-1,0,0,1,1,0,1,1,1,1,1,1,1,-1,-1,0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,-1,0,1,1,1,-1,0,-1,1,1,1,-1,-1,0,-1,-1,1,1,0,1,1,1,1,-1,1,1,1,1,-1,1,-1
5191,1,1,1,-1,1,-1,1,-1,1,1,-1,1,-1,-1,-1,-1,1,1,0,1,1,1,-1,-1,1,-1,1,1,0,1,-1
5390,-1,1,1,-1,1,-1,0,-1,1,1,1,1,1,-1,-1,-1,1,1,0,1,1,1,-1,1,1,0,1,1,0,1,-1
860,1,-1,1,1,1,-1,1,1,1,-1,-1,1,-1,0,0,-1,-1,1,0,-1,1,-1,-1,1,1,-1,-1,1,0,-1,1


In [15]:
os.makedirs("output",exist_ok=True)

In [16]:
trainX.to_csv("output/train-V-1.csv",index=False)
testX.to_csv("output/test-V-1.csv",index=False)

In [17]:
bucket

'threat-matrix-sagemaker-bucket'

In [18]:
### send data to s3  
sk_prefix="sagemaker/phishing_classification/sklearncontainer"

trainpath=session.upload_data(path="output/train-V-1.csv",bucket=bucket,key_prefix=sk_prefix)
testpath=session.upload_data(path="output/test-V-1.csv",bucket=bucket,key_prefix=sk_prefix)
print(trainpath)
print(testpath)

s3://threat-matrix-sagemaker-bucket/sagemaker/phishing_classification/sklearncontainer/train-V-1.csv
s3://threat-matrix-sagemaker-bucket/sagemaker/phishing_classification/sklearncontainer/test-V-1.csv


## Script used for training - sagemaker

In [19]:
%%writefile train.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score
import pandas as pd
import numpy as np
import sklearn
import joblib
import boto3
from io  import StringIO
import pathlib
import argparse
import os

def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir,"model.joblib"))
    return clf


if __name__=="__main__":

    print("Extracting argument")
    parser=argparse.ArgumentParser()

    ## hyperparameter

    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--max_depth",type=int,default=6)
    parser.add_argument("--min_samples_split",type=int,default=2)
    parser.add_argument("--random_state",type=int,default=42)

    ## Data,model, and output directories
    parser.add_argument("--model-dir",type=str,default=os.environ.get("SM_MODEL_DIR")) ## defined in sagemaker
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test",type=str,default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file",type=str,default="train-V-1.csv")
    parser.add_argument("--test-file",type=str,default="test-V-1.csv")


    args,_=parser.parse_known_args()

    print("Sklearn version:",sklearn.__version__)
    print("joblib version",joblib.__version__)

    print("[INFO] reading data")
    print()
    train_df=pd.read_csv(os.path.join(args.train,args.train_file))
    test_df=pd.read_csv(os.path.join(args.test,args.test_file))


    features=list(train_df.columns)
    label=features.pop(-1)

    X_train=train_df[features]
    y_train=train_df[label]

    X_test=test_df[features]
    y_test=test_df[label]

    print("[INFO] training model")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 max_depth=args.max_depth,min_samples_split=args.min_samples_split,
                                 n_jobs=-1,verbose=2)
    
    model.fit(X_train,y_train)
    

    print()

    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    print("[INFO] saving model")
    print()

    y_pred_test=model.predict(X_test)
    test_accuracy=accuracy_score(y_test,y_pred_test)
    test_roc=roc_auc_score(y_test,y_pred_test)
    test_rep=classification_report(y_test,y_pred_test)
    test_cm=confusion_matrix(y_test,y_pred_test)

    print("------Metrics------")
    print("Test Accuracy:",test_accuracy)
    print("Test ROC:",test_roc)
    print("Test Report:\n",test_rep)
    print("Test Confusion Matrix:\n",test_cm)


Overwriting train.py


### AWS Sagemaker Entry Point to execute the training script

In [20]:
from sagemaker.sklearn.estimator import SKLearn
from dotenv import load_dotenv
import os

load_dotenv()

FRAMEWORK_VERSION = "0.23-1"
PYTHON_VERSION = "py3" 

role = os.getenv("SAGEMAKER_ARN")

sklearn_estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    py_version=PYTHON_VERSION,
    base_job_name="RF-sklearn-job",
    hyperparameters={
        "n_estimators": 100,
        "max_depth": 6,
        "min_samples_split": 2,
        "random_state": 42
    },
    use_spot_instances=True,
    max_wait=3600,
    max_run=3000
)


In [21]:
sklearn_estimator.fit({'train':trainpath,'test':testpath},wait=True)

[04/17/25 20:01:54] INFO     SageMaker Python SDK will  telemetry_logging.py:91
                             collect telemetry to help                         
                             us better understand our                          
                             user's needs, diagnose                            
                             issues, and deliver                               
                             additional features.                              
                             To opt out of telemetry,                          
                             please disable via                                
                             TelemetryOptOut parameter                         
                             in SDK defaults config.                           
                             For more information,                             
                             refer to                                          
                             https://sag

####  to check artifacts from s3

In [22]:
sklearn_estimator.latest_training_job.wait(logs=False)
artifact=sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)

print(artifact["ModelArtifacts"]["S3ModelArtifacts"])


2025-04-17 14:34:30 Starting - Preparing the instances for training
2025-04-17 14:34:30 Downloading - Downloading the training image
2025-04-17 14:34:30 Training - Training image download completed. Training in progress.
2025-04-17 14:34:30 Uploading - Uploading generated training model
2025-04-17 14:34:30 Completed - Training job completed
s3://sagemaker-us-east-1-888577032243/RF-sklearn-job-2025-04-17-14-31-54-540/output/model.tar.gz


### Deploying the model for endpoint

In [30]:
from sagemaker.sklearn import SKLearnModel
from time import gmtime, strftime
from dotenv import load_dotenv

load_dotenv()
role=os.getenv("SAGEMAKER_ARN")

model_name="RF-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S",gmtime())
model=SKLearnModel(
    name=model_name,
    model_data=sklearn_estimator.model_data,
    role=role,
    entry_point="train.py",
    framework_version=FRAMEWORK_VERSION,
    py_version=PYTHON_VERSION
)

In [31]:
model

In [29]:
sklearn_estimator.model_data


's3://sagemaker-us-east-1-888577032243/RF-sklearn-job-2025-04-17-14-31-54-540/output/model.tar.gz'

In [32]:
## endpoint deployment
from time import gmtime, strftime

endpoint_name = "RF-sklearn-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint Name: {}".format(endpoint_name))


predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)



Endpoint Name: RF-sklearn-endpoint-2025-04-17-14-44-34
[04/17/25 20:14:40] INFO     Creating model with name:          session.py:4094
                             RF-sklearn-model-2025-04-17-14-44-                
                             16                                                
[04/17/25 20:14:41] INFO     Creating endpoint-config with name session.py:6019
                             RF-sklearn-endpoint-2025-04-17-14-                
                             44-34                                             
[04/17/25 20:14:42] INFO     Creating endpoint with name        session.py:4841
                             RF-sklearn-endpoint-2025-04-17-14-                
                             44-34                                             
-----!

In [34]:
testX[features][0:3]

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,port,HTTPS_token,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Submitting_to_email,Abnormal_URL,Redirect,on_mouseover,RightClick,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report
10582,-1,-1,1,1,1,-1,-1,0,-1,1,1,1,-1,-1,-1,-1,1,1,0,1,1,1,1,1,1,0,-1,1,1,1
7533,-1,-1,1,1,1,-1,0,-1,-1,1,1,1,1,0,-1,-1,-1,1,1,1,1,1,1,-1,1,1,-1,1,-1,1
10806,-1,1,1,1,1,-1,1,-1,1,1,1,1,1,0,0,-1,1,1,0,1,1,1,1,1,1,0,1,-1,1,1


In [36]:
print(predictor.predict(testX[features][0:3]).astype(int).tolist())

[-1, -1, -1]


checked with first three values absolutely correct 

In [37]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c2c424cb-0065-4f87-ba3a-2f34e168ba7f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2c424cb-0065-4f87-ba3a-2f34e168ba7f',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 17 Apr 2025 14:58:26 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}